In [ ]:
import pandas as pd
import re 
import matplotlib.pyplot as plt 
import numpy as np

## Importing Code Files

In [ ]:
FAO_Codes=pd.read_csv('../Data/Food/FAO_codes_latest.csv')
FAO_Codes=FAO_Codes.drop_duplicates()
FAO_Codes=FAO_Codes.drop_duplicates(subset=['Country Code','ISO3 Code'])
FAO_Codes=FAO_Codes.loc[~FAO_Codes['ISO3 Code'].isin(['X06','X01','X21'])]
FAO_Codes = FAO_Codes.fillna(0)
FAO_Codes['M49 Code']=FAO_Codes['M49 Code'].astype(int)
FAO_Codes

In [ ]:
Region=pd.read_excel('../Data/Food/Food  Waste Data Set - Copy.xlsx',sheet_name=2)
Region

## Processing Files for Emission due to Solid Food waste

In [ ]:
CO2_Total_waste=pd.read_csv('../Data/Food/Emissions_Agriculture_Waste_Disposal_E_All_Data_NOFLAG.csv',encoding="ISO-8859-1",nrows=5310)
CO2_Total_waste

In [ ]:
CO2_Total_waste=CO2_Total_waste.groupby(['Area Code','Area','Item','Unit']).max().reset_index()

In [ ]:
CO2_Total_waste

In [ ]:
Solid_waste=CO2_Total_waste.loc[CO2_Total_waste['Item']=='Solid food waste'].reset_index(drop=True)
Solid_waste

In [ ]:
Solid_waste

In [ ]:
Solid_waste_Co2=Solid_waste.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')


In [ ]:
Solid_waste_Co2

In [ ]:
Solid_waste_Income_group=Solid_waste_Co2.groupby(['Income group'])['Y2019'].sum().reset_index()
Solid_waste_Income_group['Percentage'] = round(Solid_waste_Income_group['Y2019'] / 
                  Solid_waste_Income_group['Y2019'].sum(),2)*100
#Solid_waste_Income_group.drop(Solid_waste_Income_group.index, inplace=True)
Solid_waste_Income_group

## Processing files for Solid Food waste  from different sectors

In [ ]:
house_waste=pd.read_excel('../Data/Food/FWD.xlsx',sheet_name=4,header=1,skipfooter=1,dtype={'M49 code': np.int32})
Food_Service_waste=pd.read_excel('../Data/Food/FWD.xlsx',sheet_name=5,header=1,skipfooter=1,dtype={'M49 code': np.int32})
Retail_waste=pd.read_excel('../Data/Food/FWD.xlsx',sheet_name=6,header=1,skipfooter=3,dtype={'M49 code': np.int32})

In [ ]:
house_waste=house_waste.loc[house_waste['Confidence in estimate'].isin(['High confidence','Medium confidence'])]
Food_Service_waste=Food_Service_waste.loc[Food_Service_waste['Confidence in estimate'].isin(['High Confidence','Medium Confidence'])]
Retail_waste=Retail_waste.loc[Retail_waste['Confidence in estimate'].isin(['High Confidence','Medium Confidence'])]

In [ ]:
house_waste=house_waste.merge(FAO_Codes[['M49 Code','ISO3 Code']],how='left',left_on='M49 code',right_on='M49 Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')

In [ ]:
house_waste.head(1)

In [ ]:
house_waste=house_waste[['Code','Region_y','Income group','Country','Household estimate (kg/capita/year)']]

In [ ]:
house_waste=house_waste.rename(columns={'Household estimate (kg/capita/year)':'Value'})
house_waste['Value']=house_waste['Value'].astype(float)
house_waste['Category']='House_waste'
house_waste_Income_group=house_waste.groupby(['Region_y','Income group','Category','Country'])['Value'].sum().reset_index()
#house_waste_Income_group['Percentage'] = round(100 * house_waste_Income_group['Value'] / house_waste_Income_group.groupby('Income group')['Value'].transform('sum'),2)
house_waste_Income_group

In [ ]:
Food_Service_waste=Food_Service_waste.merge(FAO_Codes[['M49 Code','ISO3 Code']],how='left',left_on='M49 code',right_on='M49 Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')
Food_Service_waste.head(1)

In [ ]:
Food_Service_waste=Food_Service_waste[['Code','Region_y','Income group','Country','Food service estimate (kg/capita/year)']]
Food_Service_waste=Food_Service_waste.rename(columns={'Food service estimate (kg/capita/year)':'Value'})
Food_Service_waste['Value']=Food_Service_waste['Value'].astype(float)
Food_Service_waste['Category']='Food_Service_waste'
Food_Service_waste_Income_group=Food_Service_waste.groupby(['Region_y','Income group','Category','Country'])['Value'].sum().reset_index()
#Food_Service_waste_Income_group['Percentage'] = round(100 * Food_Service_waste_Income_group['Value'] / Food_Service_waste_Income_group.groupby('Income group')['Value'].transform('sum'),2)
Food_Service_waste_Income_group



In [ ]:
Retail_waste=Retail_waste.merge(FAO_Codes[['M49 Code','ISO3 Code']],how='left',left_on='M49 code',right_on='M49 Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')
Retail_waste

In [ ]:
Retail_waste=Retail_waste[['Code','Region_y','Income group','Country','Retail estimate (kg/capita/year)']]
Retail_waste=Retail_waste.rename(columns={'Retail estimate (kg/capita/year)':'Value'})
Retail_waste['Value']=Retail_waste['Value'].astype(float)
Retail_waste['Category']='Retail_waste'
Retail_waste_Income_group=Retail_waste.groupby(['Region_y','Income group','Category','Country'])['Value'].sum().reset_index()
#Retail_waste_Income_group['Percentage'] = round(100 * Retail_waste_Income_group['Value'] / Retail_waste_Income_group.groupby('Income group')['Value'].transform('sum'),2)
Retail_waste_Income_group

In [ ]:
Waste=pd.concat([house_waste_Income_group,Food_Service_waste_Income_group,Retail_waste_Income_group])

In [ ]:
Waste['Percentage'] = round(100 * Waste['Value'] / Waste.groupby('Income group')['Value'].transform('sum'),2)
Waste

In [ ]:
Waste.loc[Waste['Income group']=='High income']

In [ ]:
Waste_Category=Waste.groupby(['Income group','Category'])['Value'].sum().reset_index()
Waste_Category['Percentage'] = round(100 * Waste_Category['Value'] / Waste_Category.groupby('Income group')['Value'].transform('sum'),2)

Waste_Category
#Waste_Category.drop(Waste_Category.index, inplace=True)


## Processing files to calculate Emission by Food Stages

In [ ]:
Emmissions=pd.read_csv('../Data/Food/Emissions_Totals_E_All_Data_NOFLAG.csv',encoding="ISO-8859-1")
Emmissions

In [ ]:
Farm_gate_Emmissions=Emmissions.groupby(['Area Code','Area','Item','Unit']).sum().reset_index()

In [ ]:
Farm_gate_Emmissions.Unit.unique()

In [ ]:
Farm_gate_Emmissions

In [ ]:
Farm_gate_Emmissions['Item'].unique()

In [ ]:
Farm_gate_Emmissions=Farm_gate_Emmissions.loc[Farm_gate_Emmissions['Item'].isin(['Burning - Crop residues','Crop Residues',
                                                            
                                                            'Enteric Fermentation',
                                                            'Manure Management',
                                                            'Manure applied to Soils',
                                                            'Manure left on Pasture',
                                                            'On-farm energy use',
                                                            'Synthetic Fertilizers',
                                                            'Rice Cultivation'
                                                           ])]

In [ ]:
Farm_gate_Emmissions.tail(100)

In [ ]:
Farm_unwanted=Farm_gate_Emmissions.loc[Farm_gate_Emmissions['Area Code'].astype(str).str.len() > 3]

In [ ]:
Farm_unwanted

In [ ]:
Farm_gate_Emmissions=Farm_gate_Emmissions.drop(Farm_gate_Emmissions.loc[5199:6111].index,axis=0)

In [ ]:
Farm_gate_Emmissions

In [ ]:
Land_Use_Emmissions=Emmissions.loc[Emmissions['Item'].isin(['Land Use change','Emissions on agricultural land','Drained organic soils'])]

In [ ]:
Land_Use_Emmissions.loc[Land_Use_Emmissions['Area Code'].astype(str).str.len() > 3]

In [ ]:
Land_Use_Emmissions['Item'].unique()

In [ ]:
Land_Use_Emmissions=Land_Use_Emmissions.drop(Land_Use_Emmissions.loc[31249:35756].index,axis=0)

In [ ]:
Transport=pd.read_csv('../Data/Food/Environment_Transport_E_All_Data_NOFLAG.csv',encoding="ISO-8859-1")


In [ ]:
Transport=Transport.groupby(['Area Code','Area','Item','Unit']).sum().reset_index()
Transport

In [ ]:
Transport.loc[Transport['Area Code'].astype(str).str.len() > 3]

In [ ]:
Transport.drop(Transport.index[5000:5874], inplace=True)

In [ ]:
Transport

In [ ]:
Farm_gate_Emmissions=Farm_gate_Emmissions.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')

In [ ]:
Farm_gate_Emmissions.dtypes

In [ ]:
Farm_gate_Emmissions.head()

In [ ]:
Land_Use_Emmissions=Land_Use_Emmissions.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')

In [ ]:
Farm_gate_Emmissions.to_csv('../Data/Food/Tableau/Farm_gate_Emmissions.csv')

In [ ]:
Land_Use_Emmissions.to_csv('../Data/Food/Tableau/Land_Use_Emmissions.csv')

In [ ]:
Transport=Transport.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')

In [ ]:

CO2_Total_waste=CO2_Total_waste.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')

In [ ]:
Transport.to_csv('../Data/Food/Tableau/Transport.csv')

In [ ]:
CO2_Total_waste.to_csv('../Data/Food/Tableau/CO2_Total_waste.csv')

In [ ]:
Transport.head(1)

In [ ]:
Farm_gate_Emmissions.dtypes

In [ ]:
Farm=Farm_gate_Emmissions.groupby(['Region','Area','Item','Income group','Unit','Area Code'])['Y2019'].mean().reset_index()

In [ ]:
Farm['Category']='Agriculture'

In [ ]:
Land=Land_Use_Emmissions.groupby(['Region','Area','Item','Income group','Unit','Area Code'])['Y2019'].mean().reset_index()

In [ ]:
Land['Category']='Land_Use_Change'

In [ ]:
agri_transport=Transport.groupby(['Region','Area','Item','Income group','Unit','Area Code'])['Y2019'].mean().reset_index()

In [ ]:
agri_transport['Category']='Transport'

In [ ]:
agri_Co2_waste=CO2_Total_waste.groupby(['Region','Area','Item','Income group','Unit','Area Code'])['Y2019'].mean().reset_index()

In [ ]:
agri_Co2_waste['Category']='Food_Waste'

In [ ]:
agri_Co2_waste['Item'].unique()

In [ ]:
agri_Co2_waste=agri_Co2_waste.loc[agri_Co2_waste['Item']!='Total  waste from food systems']

In [ ]:
type(Land)

In [ ]:
Total_Agri_Emmision=pd.concat([Farm,Land,agri_transport,agri_Co2_waste])

In [ ]:
Item_Area_Region=Total_Agri_Emmision.groupby(['Region','Area','Category','Item'])['Y2019'].sum().reset_index()
Item_Area_Region['Percentage']= round(100 * Item_Area_Region['Y2019'] / Item_Area_Region.groupby('Area')['Y2019'].transform('sum'),2)
Item_Area_Region

In [ ]:
Item_Area_Region.loc[Item_Area_Region['Area']=='Zimbabwe']

In [ ]:
Area_Category_Region=Total_Agri_Emmision.groupby(['Region','Category','Area'])['Y2019'].sum().reset_index()
Area_Category_Region['Percentage'] = round(100 * Area_Category_Region['Y2019'] / Area_Category_Region.groupby('Category')['Y2019'].transform('sum'),2)
Area_Category_Region

In [ ]:
Item_Category_Region=Total_Agri_Emmision.groupby(['Region','Category','Item'])['Y2019'].sum().reset_index()
Item_Category_Region['Percentage'] = round(100 * Item_Category_Region['Y2019'] / Item_Category_Region.groupby('Category')['Y2019'].transform('sum'),2)
Item_Category_Region

In [ ]:
Item_Category_Region.to_csv('../Data/Food/Tableau/Item_Category_Region.csv')

In [ ]:
Stage=Total_Agri_Emmision.groupby('Category')['Y2019'].mean().reset_index()
Stage

In [ ]:
Stage.to_csv('../Data/Food/Tableau/Stage.csv')

In [ ]:
Food_Emission_Region=Total_Agri_Emmision.groupby('Region')['Y2019'].sum().reset_index()
Food_Emission_Region['Percentage']=round(Food_Emission_Region['Y2019'] / 
                  Food_Emission_Region['Y2019'].sum(),2)*100
Food_Emission_Region

In [ ]:
region_catgry_emission=Total_Agri_Emmision.groupby(['Region','Category'])['Y2019'].mean().reset_index()


region_catgry_emission['Percentage'] = round(100 * region_catgry_emission['Y2019'] / region_catgry_emission.groupby('Region')['Y2019'].transform('sum'),2)
region_catgry_emission

## Processing files for Food Production

In [ ]:
Food_production=pd.read_csv('../Data/Food/FAO_food_production.csv')
Food_production

In [ ]:
Food_production=Food_production.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code (FAO)',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')


In [ ]:
Food_production

In [ ]:
Food_production['Region'].unique()

In [ ]:
Food_prod_Region=Food_production.groupby('Region')['Value'].sum().reset_index()
Food_prod_Region['Percentage']=round(Food_prod_Region['Value'] / 
                  Food_prod_Region['Value'].sum(),2)*100
Food_prod_Region

## Processing files for undernourished

In [ ]:
undernourished=pd.read_csv('../Data/Food/FAOSTAT_undernourished.csv')

In [ ]:
undernourished['Year'].unique()

In [ ]:
undernourished.loc[undernourished['Area Code'].astype(str).str.len() > 3]


In [ ]:
undernourished.drop(undernourished.index[3876:5675], inplace=True)

In [ ]:
undernourished=undernourished.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')


In [ ]:
undernourished=undernourished.loc[undernourished['Year']=='2018-2020']

In [ ]:
undernourished

In [ ]:
Population=pd.read_excel('../Data/Food/Food  Waste Data Set - Copy.xlsx',sheet_name=3)

In [ ]:
Population

In [ ]:
Population=Population.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code (FAO)',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')
Population

In [ ]:
Population=Population.loc[Population['Element']=='Total Population - Both sexes']

In [ ]:
Population

In [ ]:
undernourish_pop=undernourished.merge(Population[['Area Code (FAO)','Value','Unit']],how='inner',left_on='Country Code',right_on='Area Code (FAO)')

In [ ]:
undernourish_pop.dtypes

In [ ]:
undernourish_pop.loc[undernourish_pop['Value_x']=='0.0']

In [ ]:
undernourish_pop['Value_x'].value_counts()

In [ ]:
for index,rows in undernourish_pop.iterrows():
    if rows['Value_x']== '<0.1':
        undernourish_pop.loc[index,'Value_x']='0.09'


In [ ]:

#undernourished_pop['Value_x'].unique()

undernourish_pop['Value_x']=undernourish_pop['Value_x'].astype(float)
undernourish_pop['Value_y']=undernourish_pop['Value_y'].astype(float)

In [ ]:
undernourish_pop=undernourish_pop[['Area Code','Area','Unit_x','Value_x','Value_y','Unit_y','Region','Code','Income group']].fillna(0)

In [ ]:
undernourish_pop

In [ ]:
round(((undernourish_pop['Value_x']*1000000)/(undernourish_pop['Value_y']*1000))*100,2)

In [ ]:
undernourish_pop['Perc_undernourished']=round(((undernourish_pop['Value_x']*1000000)/(undernourish_pop['Value_y']*1000))*100,2)

In [ ]:
undernourish_pop

In [ ]:
undernourish_pop.loc[undernourish_pop['Area']=='Zambia']

In [ ]:
undernourish_pop.loc[undernourish_pop['Perc_undernourished']==0.0]

In [ ]:
undernourish_pop['Perc_undernourished'].unique()

Food_production.loc[Food_production['Code']=='DZA']

## Processing files for Temperature Change

In [ ]:
temp_change=pd.read_csv('../Data/Food/Environment_Temperature_change_E_All_Data_(Normalized).csv',encoding="ISO-8859-1")

In [ ]:
temp_change

In [ ]:
temp_change.loc[temp_change['Area Code'].astype(str).str.len() > 3]



In [ ]:
temp_change.drop(temp_change.index[467976:544714], inplace=True)

In [ ]:
temp_change=temp_change.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')

## Processing files for caluculating Emission share of Food Systems

In [ ]:
Emission_share=pd.read_csv('../Data/Food/Environment_Emissions_by_Sector_E_All_Data_(Normalized).csv',encoding="ISO-8859-1")


In [ ]:
Emission_share['Item'].unique()

In [ ]:
Emission_share=Emission_share.loc[Emission_share['Item'].isin(['Farm-gate emissions',
                                                               'Land Use change',
                                                               "Pre- and post- production",
                                                               'On-farm electricity use',
                                                              'Emissions on agricultural land',
                                                              "Energy-food systems"])]

In [ ]:
Emission_share.loc[Emission_share['Area Code'].astype(str).str.len() > 3]


In [ ]:
Emission_share=Emission_share.drop(Emission_share.loc[1402508:1683948].index,axis=0)

In [ ]:
Emission_share

In [ ]:
Emission_share=Emission_share.loc[Emission_share['Unit']!='kilotonnes']

In [ ]:
Emission_share

In [ ]:
Emission_share=Emission_share.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')

In [ ]:
Emission_share[(Emission_share['Area']=='Zimbabwe') & (Emission_share['Year']==2019)]

In [ ]:
Emission_share=Emission_share.groupby(['Area Code','Area','Item Code','Item','Year Code','Unit','Code','Region','Income group']).mean(['Value']).reset_index()

In [ ]:
Emission_share

In [ ]:
Food_system_share=Emission_share.groupby(['Area Code','Area','Year Code','Unit','Code','Region','Income group']).mean(['Value']).reset_index()

In [ ]:
Food_system_share

In [ ]:
Food_system_share['Non-Food Emissions']=100-Food_system_share['Value']

In [ ]:
Food_system_share=Food_system_share.rename(columns={'Value':'Food_Emission'})

In [ ]:
Food_system_share

## Dashboard2 file

In [ ]:
Food_system_share.to_csv('../Data/Food/Tableau/Food_system_share.csv')


## Dashboard2 file

In [ ]:
Food_prod_Region.to_csv('../Data/Food/Tableau/Food_prod_Region.csv')

## Dashboard2 file

In [ ]:
region_catgry_emission.to_csv('../Data/Food/Tableau/region_catgry_emission.csv')

## Dashboard3 file

In [ ]:
temp_change.to_csv('../Data/Food/Tableau/temp_change.csv')

## Dashboard3 file

In [ ]:
Area_Category_Region.to_csv('../Data/Food/Tableau/Area_Category_Region.csv')

## Dashboard3 file

In [ ]:
Item_Area_Region.to_csv('../Data/Food/Tableau/Item_Area_Region.csv')

## Dashboard 4 file

In [ ]:
undernourish_pop.to_csv('../Data/Food/Tableau/undernourished.csv')

## Dashboard4 file

In [ ]:
Waste.to_csv('../Data/Food/Tableau/Waste.csv')
Waste_Category.to_csv('../Data/Food/Tableau/Waste_Category.csv')


## Dashboard4 file

In [ ]:
Solid_waste_Income_group.to_csv('../Data/Food/Tableau/Solid_waste_Income_group.csv')

## Dashboard1 file

In [ ]:
Food_Emission_Region.to_csv('../Data/Food/Tableau/Food_Emission_Region.csv')

In [ ]:
Solid_waste_Co2.to_csv('../Data/Food/Tableau/Solid_waste_Co2.csv')

## Identify countries not mapped to Regions

In [ ]:
NULL_regions=Solid_waste_Co2.loc[Solid_waste_Co2['ISO3 Code'].isna()]

In [ ]:
NULL_regions.to_csv('../Data/Food/NULLRegions.csv')

## Codes written prior to finalised version 

Agri_share.to_csv('../Data/Food/Tableau/Agri_share.csv')

Agri_share

Agri_share=Agri_share.melt(id_vars=["Area Code","Area","Year Code","Unit","Code","Region","Income group","Item Code","Element Code","Year"], 
        var_name="Emission Category", 
        value_name="Value")

Agri_share[(Agri_share['Area']=='Zimbabwe') & (Agri_share['Year']==2019)]

Agri_share=Agri_share.loc[Agri_share['Emission Category']!='Country Code']

Agri_share


Agri_share['Value']=Agri_share['Value'].apply(lambda x: str(round(x,2)) + '%')


Agri_share

Agri_share.groupby('Region')['Value'].mean().reset_index()

Agri_share_region=Agri_share.groupby('Region')['Value'].sum().reset_index()
Agri_share_region['Percentage']=round(Agri_share_region['Value'] / 
                  Agri_share_region['Value'].sum(),2)*100
Agri_share_region

Agri_share_region.to_csv('../Data/Food/Tableau/Agri_share_region.csv')

Agri_share.dtypes



Food_share=pd.read_csv('../Data/Food/Environment_Emissions_by_Sector_E_All_Data_(Normalized).csv',encoding="ISO-8859-1")
Food_share=Food_share.loc[Food_share['Unit']=='kilotonnes']

Food_share.loc[Food_share['Area Code'].astype(str).str.len() > 3]


Food_share.loc[Food_share['Value'] < 0]

Food_share=Food_share.drop(Food_share.loc[1402508:1683798].index,axis=0)

Food_share.index

Food_share

Food_share=Food_share.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')

afghan=Food_share.loc[(Food_share['Year']==2019)&(Food_share['Area']=='Afghanistan')]

afghan.to_csv('../Data/Food/afghan.csv')

Food_share['Item'].unique()

Land_Use_Share=Food_share.loc[Food_share['Item']=='Land Use change']

Land_Use_Share

Land_Use_Share.loc[(Land_Use_Share['Year']==2019)&(Land_Use_Share['Area']=='Zimbabwe')]

Land_Use_Share=Land_Use_Share.groupby(['Area Code','Area','Item Code','Item','Year','Code','Unit','Region','Income group'])['Value'].sum('Value').reset_index()

Land_Use_Share=Land_Use_Share.rename(columns={'Value':'Land_use_share'})

Land_Use_Share

Land_Use_Share.groupby('Region')['Land_use_share'].mean('Land_use_share')

Agri_Prod_share=Food_share.loc[Food_share['Item'].isin(['Farm-gate emissions','Fertilizers Manufacturing','Emissions on agricultural land'])]

Agri_Prod_share

Agri_Prod_share=Agri_Prod_share.groupby(['Area Code','Area','Item Code','Item','Year','Unit','Code','Region','Income group'])['Value'].sum('Value').reset_index()

Agri_Prod_share

Agri_Prod_share.loc[(Agri_Prod_share['Year']==2019)&(Agri_Prod_share['Area']=='Zimbabwe')]

Agri_total=Agri_Prod_share.groupby(['Area Code','Area','Year','Unit','Code','Region','Income group'])['Value'].mean('Value').reset_index()

Agri_total

Agri_total.groupby('Region')['Agri_share'].mean()

Agri_total=Agri_total.rename(columns={'Value':'Agri_share'})

Food_Supply=Food_share.loc[Food_share['Item'].isin(['Food Processing','Food Transport','Food Retail','Food Packaging'])]

Food_Supply=Food_Supply.groupby(['Area Code','Area','Item Code','Item','Year','Unit','Code','Region','Income group'])['Value'].sum('Value').reset_index()

Supply_share=Food_Supply.groupby(['Area Code','Area','Year','Unit','Code','Region','Income group'])['Value'].mean('Value').reset_index()

Supply_share

Supply_share=Supply_share.rename(columns={'Value':'Supply_share'})

Post_retail=Food_share.loc[Food_share['Item'].isin(['Food Waste Disposal','Food Household Consumption'])]


Post_Retail=Post_retail.groupby(['Area Code','Area','Item Code','Item','Year','Unit','Code','Region','Income group'])['Value'].sum('Value').reset_index()

Post_prod=Post_Retail.groupby(['Area Code','Area','Year','Unit','Code','Region','Income group'])['Value'].mean('Value').reset_index()

Post_prod

Post_prod=Post_prod.rename(columns={'Value':'Post_prod'})

Post_prod

Land_Use_Share[['Area Code','Land_use_share']]
Agri_total[['Area Code','Agri_share']]
Supply_share[['Area Code','Supply_share']]
Post_prod[['Area Code','Post_prod']]

Land_Use_Share.to_csv('../Data/Food/Tableau/Land_Use_Share.csv')
Agri_total.to_csv('../Data/Food/Tableau/Agri_total.csv')
Supply_share.to_csv('../Data/Food/Tableau/Supply_share.csv')
Post_prod.to_csv('../Data/Food/Tableau/Post_prod.csv')

Land_2019=Land_Use_Share.loc[Land_Use_Share['Year']==2019]

Land_2019.groupby('Region').sum()


Agri_2019=Agri_total.loc[Agri_total['Year']==2019]
Supply_2019=Supply_share.loc[Supply_share['Year']==2019]
Post_2019=Post_prod.loc[Post_prod['Year']==2019]


Supply_2019.loc[Supply_2019['Code']=='CHN']

Food_System_Emission=Land_2019.merge(Agri_2019[['Area Code','Agri_share']],how='inner',on='Area Code').merge(Supply_2019[['Area Code','Supply_share']],how='inner',on='Area Code').merge(Post_2019[['Area Code','Post_prod']],how='inner',on='Area Code')

Food_System_Emission

Food_System_Emission=Food_System_Emission.melt(id_vars=["Area Code","Area","Year","Unit","Code","Item Code","Item","Region","Income group"], 
        var_name="Emission Category", 
        value_name="Value")

Food_System_Emission.to_csv('../Data/Food/Tableau/Food_System_Emission.csv')

Land_Forest=pd.read_csv('../Data/Food/Emissions_Land_Use_Forests_E_All_Data_NOFLAG.csv',encoding="ISO-8859-1")
Land_Fires=pd.read_csv('../Data/Food/Emissions_Land_Use_Fires_E_All_Data_NOFLAG.csv',encoding="ISO-8859-1")
Land_Drained_organic_Soils=pd.read_csv('../Data/Food/Emissions_Drained_Organic_Soils_E_All_Data_NOFLAG.csv',encoding="ISO-8859-1")


Land_Forest.loc[Land_Forest['Area Code'].astype(str).str.len() > 3]

Land_Forest.drop(Land_Forest.index[1383:1587], inplace=True)

Land_Forest

Land_Fires.loc[Land_Fires['Area Code'].astype(str).str.len() > 3]

Land_Fires.drop(Land_Fires.index[10697:12325], inplace=True)

Land_Drained_organic_Soils.loc[Land_Drained_organic_Soils['Area Code'].astype(str).str.len() > 3]

Land_Drained_organic_Soils.drop(Land_Drained_organic_Soils.index[1443:1863], inplace=True)

Land_Drained_organic_Soils

Land_Drained_organic_Soils=Land_Drained_organic_Soils.loc[Land_Drained_organic_Soils['Unit']=='kilotonnes']
Land_Fires=Land_Fires.loc[Land_Fires['Unit']=='kilotonnes']
Land_Forest=Land_Forest.loc[Land_Forest['Unit']=='kilotonnes']

Land_Drained_organic_Soils=Land_Drained_organic_Soils.loc[Land_Drained_organic_Soils['Element Code']!=7216]

Land_Drained_organic_Soils=Land_Drained_organic_Soils.groupby(['Area Code','Area','Item','Unit']).sum().reset_index()
Land_Drained_organic_Soils

Land_Fires=Land_Fires.groupby(['Area Code','Area','Item','Unit']).sum().reset_index()
Land_Fires

Land_Forest

import requests
import json

endpoint='https://api.data.apps.fao.org/api/v2/bigquery?sql_url=https://data.apps.fao.org/catalog/dataset/86b7daf0-4fae-450b-b88f-76690e2fdc17/resource/0026de2e-c12c-457c-b4f5-8c4cd2ef7960/download/crop-calendar-parameterized-query.sql'

response=requests.get(endpoint)

response.json()


Farm_Item=Farm.groupby('Item')['Y2019'].mean().reset_index()




Land_Fires=Land_Fires.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')
Land_Forest=Land_Forest.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')


Land_Drained_organic_Soils=Land_Drained_organic_Soils.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')

Land_Drained_organic_Soils.to_csv('../Data/Food/Tableau/Land_Drained_organic_Soils.csv')

Land_Fires.to_csv('../Data/Food/Tableau/Land_Fires.csv')
Land_Forest.to_csv('../Data/Food/Tableau/Land_Forest.csv')



Hunger_index=pd.read_csv('../Data/Food/global-hunger-index-vs-gdp-per-capita.csv')
Hunger_2021=Hunger_index.loc[Hunger_index['Year']==2021]
Hunger_2021=Hunger_2021.loc[~((Hunger_2021['Code'].isna()) | (Hunger_2021['Code']=='OWID_WRL'))]
Hunger_2021=Hunger_2021.merge(Region[['Code','Region','Income group']],how='inner',left_on='Code',right_on='Code')
Hunger_2021

Hunger_2021['Global Hunger Index (2021)']=Hunger_2021['Global Hunger Index (2021)'].fillna(0)

Hunger_2021['Region'].unique()

Hunger_2021.to_csv('../Data/Food/Tableau/Hunger_2021.csv')

Agri_employment=pd.read_excel('../Data/Food/Food  Waste Data Set - Copy.xlsx',sheet_name=4)
Agri_Expenditure=pd.read_excel('../Data/Food/Food  Waste Data Set - Copy.xlsx',sheet_name=5)

Agri_Expenditure.dtypes

Agri_Expenditure=Agri_Expenditure.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code (FAO)',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')
Agri_emplymnt=Agri_employment.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')


Agri_emplymnt=Agri_emplymnt.loc[Agri_emplymnt['Indicator'].isin(['Employment in agriculture, forestry and fishing by age, total (15+)',
'Employment in agriculture, forestry and fishing by age, 15 to 24',
'Employment in agriculture, forestry and fishing by age, 25 to 54',
'Employment in agriculture, forestry and fishing by age, 55 to 64',
'Employment in agriculture, forestry and fishing by age, 65+',
'Mean weekly hours actually worked per employed person in agriculture, forestry and fishing',
'Mean weekly hours actually worked per employee in agriculture, forestry and fishing',
'Share of employment in agriculture, forestry and fishing in total employment',
'Share of females in total employment in agriculture, forestry and fishing',
'Share of employees in agriculture, forestry and fishing in total employees',
'Share of female employees in total employees in agriculture, forestry and fishing',
'Agriculture value added per worker (constant 2015 US$)'])]

Agri_emplymnt.to_csv('../Data/Food/Tableau/Agri_emplymnt.csv')
Agri_Expenditure.to_csv('../Data/Food/Tableau/Agri_Expenditure.csv')

Food_shortage=pd.read_csv('../Data/Food/countries-requiring-external-assistance-for-food.csv',encoding="ISO-8859-1")
Food_shortage

Food_shortage['Date'].unique()

Food_shortage.loc[Food_shortage['Date']=='2009-04']


Prod_year=pd.read_csv('../Data/Food/Production_Crops_Livestock_E_All_Data_(Normalized).csv',encoding="ISO-8859-1")
Prod_year

Prod_year=Prod_year.loc[Prod_year['Element']=='Production']

Prod_year=Prod_year.reset_index(drop=True)

Prod_year

Prod_year.loc[Prod_year['Area Code'].astype(str).str.len() > 3]


Prod_year.drop(Prod_year.index[1098190:1481937],inplace=True)

Prod_year=Prod_year.merge(FAO_Codes[['Country Code','ISO3 Code']],how='inner',left_on='Area Code',right_on='Country Code').merge(Region[['Code','Region','Income group']],how='inner',left_on='ISO3 Code',right_on='Code')

Prod_year.to_csv('../Data/Food/Tableau/Prod_year.csv')

Prod_year.loc[Prod_year['Area']=='Afghanistan']

Prod_year.groupby(['Area Code','Area','Element Code','Element','Year Code','Year','Unit']).sum('Value').reset_index()

house_waste.to_csv('../Data/Food/Tableau/House_waste.csv')
Food_Service_waste.to_csv('../Data/Food/Tableau/Food_Service_waste.csv')
Retail_waste.to_csv('../Data/Food/Tableau/Retail_waste.csv')

Food_production.to_csv('../Data/Food/Tableau/Food_production.csv')

Food_production.to_csv('../Data/Food/Tableau/Food_production.csv')



Farm_emission_trend=Farm_gate_Emmissions.groupby(["Area Code","Area","Unit","Code","Region","Income group"]).mean().reset_index()

Farm_emission_trend

Farm_emission_trend=Farm_emission_trend.melt(id_vars=["Area Code","Area","Unit","Code","Region","Income group"], 
        var_name="Year", 
        value_name="Value")

Farm_emission_trend.to_csv('../Data/Food/Tableau/Farm_emission_trend.csv')